<a href="https://colab.research.google.com/github/modi2009/ComputerVision/blob/GANS/CycleGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
suyashdamle_cyclegan_path = kagglehub.dataset_download('suyashdamle/cyclegan')

print('Data source import complete.')


In [ ]:
# import important libraries
import numpy as np
import torch
import torch.nn as nn
import torchvision.utils
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import os
from PIL import Image
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torchvision.utils import save_image


# **Discriminator**

In [ ]:
# Create Convolutional Block function
def Conv2D(in_channels, out_channels,kernel_size = 4, stride = 2, padding = 1, padding_mode = 'reflect', dropout = 0):
  layer = nn.Sequential(
      nn.Conv2d(in_channels, out_channels, kernel_size = kernel_size, stride = stride, padding = padding, padding_mode = padding_mode, bias = True),
      nn.InstanceNorm2d(out_channels),
      nn.LeakyReLU(0.2),
  )
  return layer

In [ ]:
# Create Discriminator Model
class Discriminator(nn.Module):
    def __init__(self, in_channels = 3, features = [64,128,256,512]):
        super().__init__()
        self.initial = nn.Sequential(
            nn.Conv2d(in_channels, features[0], kernel_size = 4, stride = 2, padding = 1, padding_mode = 'reflect'),
            nn.LeakyReLU(0.2),
        )

        layers = []
        in_channels = features[0]
        for feature in features[1:]:
            layer = Conv2D(in_channels, feature, stride = 1 if feature == features[-1] else 2)
            layers.append(layer)
            in_channels = feature

        layers.append(
            nn.Conv2d(
                in_channels, 1, kernel_size = 4, stride = 1, padding = 1 , padding_mode = 'reflect'
            )
        )
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        x = self.initial(x)
        x = self.model(x)
        return torch.sigmoid(x)


In [ ]:
def testDiscriminator():
    x = torch.randn((1, 3, 256, 256))
    model = Discriminator()
    for param in model.parameters():
        print(param.dtype)  # Should output torch.float16 for mixed precision
    preds = model(x)
    return preds.shape
testDiscriminator()

torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32


torch.Size([1, 1, 30, 30])

# **Generator**

In [ ]:
# Create Convolutional Transpose Block function
def ConvTranspose2D(num_channel, out_channels,kernel_size = 4, stride = 2, padding = 1):
  layer = nn.Sequential(
      nn.ConvTranspose2d(num_channel, out_channels, kernel_size = kernel_size, stride = stride, padding = padding, output_padding = 1),
      nn.InstanceNorm2d(out_channels),
      nn.ReLU(True),
  )
  return layer

In [ ]:
# Create Convolutional Block function
def Conv2DGen(in_channels, out_channels,use_act = True, kernel_size = 4, stride = 2, padding = 1, padding_mode = 'reflect', dropout = 0):
  layer = nn.Sequential(
      nn.Conv2d(in_channels, out_channels, kernel_size = kernel_size, stride = stride, padding = padding, padding_mode = padding_mode, bias = True),
      nn.InstanceNorm2d(out_channels),
      nn.ReLU(True) if use_act else nn.Identity(),
  )
  return layer

In [ ]:
def ResidualBlock(in_channels):
    layer = nn.Sequential(
        Conv2DGen(in_channels, in_channels, kernel_size = 3, padding = 1,stride = 1),
        Conv2DGen(in_channels, in_channels, use_act= False, kernel_size = 3, padding = 1,stride = 1)
    )
    return layer

In [ ]:
# Create Discriminator Model
class Generator(nn.Module):
    def __init__(self, in_channels = 3, features = [64,128], num_residuals = 9):
        super().__init__()

        # create encoder

        # create initial layer of encoder
        self.initial_down = nn.Sequential(
            nn.Conv2d(in_channels, features[0], kernel_size = 7, stride = 1, padding = 3, padding_mode = 'reflect'),
            nn.ReLU(True),
        )

        # Create down blocks
        self.layers_down = nn.ModuleList()
        for feature in features:
            layer = Conv2DGen(feature, feature*2, kernel_size = 3, padding = 1, stride = 2)
            self.layers_down.append(layer)


        # create residual block
        self.residual_blocks = nn.ModuleList(
            [ResidualBlock(features[-1]*2) for _ in range(num_residuals)]
        )

        # creatue upsampling blocks
        self.layers_up = nn.ModuleList()
        features_reversed = reversed(features)
        for feature in features_reversed:
            layer = ConvTranspose2D(feature*2, feature, kernel_size = 3, padding = 1, stride = 2)
            self.layers_up.append(layer)

        # last layer
        self.last_layer = nn.Sequential(
            nn.Conv2d(features[0], in_channels, kernel_size = 7, stride = 1, padding = 3, padding_mode = 'reflect'),
        )


    def forward(self, x):
        x = self.initial_down(x)
        for layer in self.layers_down:
            x = layer(x)
        for layer in self.residual_blocks:
            x = layer(x) + x
        for layer in self.layers_up:
            x = layer(x)
        x = self.last_layer(x)

        return torch.tanh(x)



In [ ]:
def testGenerator():
    x = torch.randn((2, 3, 256, 256))
    model = Generator()
    preds = model(x)
    return preds.shape
testGenerator()

torch.Size([2, 3, 256, 256])

# **Load and Augment Dataset**

In [ ]:
class MapDataset(Dataset):
    def __init__(self, root_zebra, root_horse, transform):
        self.root_zebra = root_zebra
        self.root_horse = root_horse
        self.transform = transform

        self.zebra_images = os.listdir(root_zebra)
        self.horse_images = os.listdir(root_horse)

        self.length_zebra = len(self.zebra_images)
        self.length_horse = len(self.horse_images)
        self.length_dataset = max(self.length_zebra, self.length_horse)


    def __len__(self):
        return self.length_dataset

    def __getitem__(self, idx):
        zebra_image = self.zebra_images[idx % self.length_zebra]
        horse_image = self.horse_images[idx % self.length_horse]

        zebra_path = os.path.join(self.root_zebra, zebra_image)
        horse_path = os.path.join(self.root_horse, horse_image)

        zebra_image = np.array(Image.open(zebra_path).convert('RGB'))
        horse_image = np.array(Image.open(horse_path).convert('RGB'))

                # Apply transformations
        if self.transform:
            augmented = self.transform(image=zebra_image, image0=horse_image)
            zebra_image = augmented["image"]
            horse_image = augmented["image0"]



        return zebra_image, horse_image


In [ ]:
transform = A.Compose([
    A.Resize(256.0,256.0),
    A.Normalize([.5,.5,.5], [.5,.5,.5], max_pixel_value = 255.0),
    ToTensorV2()
],additional_targets = {'image0':'image'})


In [ ]:
BATCH_SIZE = 4
NUM_WORKERS = 4
# Load data and create DataLoader
train_dataset = MapDataset(
    root_zebra='/kaggle/input/cyclegan/horse2zebra/horse2zebra/trainB',
    root_horse = '/kaggle/input/cyclegan/horse2zebra/horse2zebra/trainA',
    transform=transform,
)

val_dataset = MapDataset(
    root_zebra='/kaggle/input/cyclegan/horse2zebra/horse2zebra/testB',
    root_horse = '/kaggle/input/cyclegan/horse2zebra/horse2zebra/testA',
    transform=transform,
)


train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    shuffle=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    shuffle=False
)


In [ ]:
os.makedirs('images', exist_ok = True)

# **Create Loss Functions**

In [ ]:
# initialize criterion loss
criterion = nn.MSELoss()
L1_LOSS = nn.L1Loss()
# discriminator loss function
def discriminator_loss(real_preds, fake_preds):
  # initialize targets
  target_true = torch.ones_like(real_preds)
  target_false = torch.zeros_like(fake_preds)

  # compute losses
  real_loss = criterion(real_preds, target_true)
  fake_loss = criterion(fake_preds, target_false)

  return real_loss + fake_loss

def generator_loss(fake_preds, image, cycle_image, identity_image, lamda_cycle, lamdia_I):

  # initialize target
  target_true = torch.ones_like(fake_preds)

  # compute loss
  gen_loss = criterion(fake_preds, target_true)
  cycle_loss = L1_LOSS(image, cycle_image) * lamda_cycle
  identity_loss = L1_LOSS(image, identity_image) * lamdia_I
  return gen_loss + cycle_loss + identity_loss

# **Train Model**

In [ ]:
# Initialize Hyperparameters
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
LEARNING_RATE = 2e-4
IMAGE_SIZE = 256
CHANNELS_IMAGE = 3
LAMDA_CYCLE = 10
LAMDA_I = 0.0
NUM_EPOCH = 200


In [ ]:
# Initialize Models
disc_h = Discriminator(CHANNELS_IMAGE).to(DEVICE)
disc_z = Discriminator(CHANNELS_IMAGE).to(DEVICE)
gen_h = Generator(CHANNELS_IMAGE).to(DEVICE)
gen_z= Generator(CHANNELS_IMAGE).to(DEVICE)

In [ ]:
import torch.optim as optim
# Initialize Optimizer
opt_disc = optim.Adam(
    list(disc_h.parameters()) + list(disc_z.parameters()),
    lr = LEARNING_RATE,
    betas =(0.5,.999)
)

opt_gen = opt_disc = optim.Adam(
    list(gen_h.parameters()) + list(gen_z.parameters()),
    lr = LEARNING_RATE,
    betas =(0.5,.999)
)

In [ ]:
from tqdm.notebook import tqdm
def train(num_epochs,disc_h, disc_z, gen_h, gen_z, loader, opt_disc, opt_gen, lamda_cycle, lamda_I,device):
    for epoch in range(num_epochs):
        epoch_loss_disc = 0  # Accumulate discriminator loss for the epoch
        epoch_loss_gen = 0   # Accumulate generator loss for the epoch
        with tqdm(loader, desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch") as t:
            for idx, (zebra, horse) in enumerate(t):
                zebra = zebra.to(device)
                horse = horse.to(device)

                # Train Discriminator


                # Train Discriminator of horse
                fake_horse = gen_h(zebra)
                D_h_real = disc_h(horse)
                D_h_fake = disc_h(fake_horse.detach())
                D_h_loss = discriminator_loss(D_h_real, D_h_fake)

                # Train Discriminator of zebra
                fake_zebra = gen_z(horse)
                D_z_real = disc_z(zebra)
                D_z_fake = disc_z(fake_zebra.detach())
                D_z_loss = discriminator_loss(D_z_real, D_z_fake)

                D_loss = (D_h_loss + D_z_loss)/2



                opt_disc.zero_grad()
                D_loss.backward()
                opt_disc.step()
                epoch_loss_disc += D_loss.item()


                # Train Generator


                # Train Generator of horse
                gen_h_fake = disc_h(fake_horse)
                cycle_horse = gen_h(fake_zebra)
                identitiy_horse = gen_h(horse)
                gen_h_loss = generator_loss(gen_h_fake, horse, cycle_horse, identitiy_horse, lamda_cycle, lamda_I)

                # Train Generator of zebra
                gen_z_fake = disc_z(fake_zebra)
                cycle_zebra = gen_z(fake_horse)
                identitiy_zebra = gen_z(zebra)
                gen_z_loss = generator_loss(gen_z_fake, zebra, cycle_zebra, identitiy_zebra, lamda_cycle, lamda_I)

                G_loss = gen_h_loss + gen_z_loss


                opt_gen.zero_grad()
                G_loss.backward()
                opt_gen.step()
                epoch_loss_gen += G_loss.item()
                t.set_postfix(d_loss=D_loss.item(), g_loss=G_loss.item())

                if idx % 200 == 0:
                    save_image(fake_horse*0.5 + 0.5, f"/kaggle/working/images/horse_{idx}.png")
                    save_image(fake_zebra*0.5 + 0.5, f"/kaggle/working/images/zebra_{idx}.png")

        # Average loss over all batches in the epoch
        avg_loss_disc = epoch_loss_disc / len(loader)
        avg_loss_gen = epoch_loss_gen / len(loader)
        print(f"Epoch [{epoch+1}/{num_epochs}] | "f"Avg D Loss: {avg_loss_disc:.4f} | Avg G Loss: {avg_loss_gen:.4f}")




In [ ]:
train(
    NUM_EPOCH,disc_h, disc_z, gen_h, gen_z, train_loader, opt_disc, opt_gen, LAMDA_CYCLE, LAMDA_I,DEVICE
)

Epoch 1/200:   0%|          | 0/334 [00:00<?, ?batch/s]

Epoch [1/200] | Avg D Loss: 0.5398 | Avg G Loss: 5.0407


Epoch 2/200:   0%|          | 0/334 [00:00<?, ?batch/s]

Epoch [2/200] | Avg D Loss: 0.5395 | Avg G Loss: 3.9643


Epoch 3/200:   0%|          | 0/334 [00:00<?, ?batch/s]

...

Epoch [11/200] | Avg D Loss: 0.5508 | Avg G Loss: 2.4715


Epoch 12/200:   0%|          | 0/334 [00:00<?, ?batch/s]

Epoch [12/200] | Avg D Loss: 0.5523 | Avg G Loss: 2.4273


Epoch 13/200:   0%|          | 0/334 [00:00<?, ?batch/s]

KeyboardInterrupt: 